In [1]:
import json
import websocket
import time
from functions import insert_data_candlesticks,unix_to_datetime,castToDataType,datetime_to_unix
from dotenv import load_dotenv,find_dotenv
import os
import pandas as pd
from binance.client import Client
import requests
from cassandra.cluster import Cluster

In [2]:
# Define our weight variable to control API limits
current_API_weight = 0
MAX_API_WEIGHT = 5500


# Define the path to the .env file
load_dotenv(find_dotenv('secrets.env'))

api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Initialize our client
client = Client(api_key, api_secret, testnet=True)


In [3]:
# Helper function to control API limits
def check_API_limits():
    global current_API_weight
    if current_API_weight >= MAX_API_WEIGHT:
        current_time = time.localtime()
        # seconds remaining until the end of the current minute
        seconds_left = 60 - current_time.tm_sec
        print(
            f"API limit exceeded with weight = {current_API_weight}. Waiting for {seconds_left} seconds.")
        time.sleep(seconds_left)
        current_API_weight = 0


# We should add start time and end time, and limit to 1000.
        
def get_klines_data(symbol, interval,startTime,endTime,limit):
    global current_API_weight
    check_API_limits()
    url = 'https://api1.binance.com'
    api_call = f'''/api/v3/klines?symbol={symbol}&interval={interval}&startTime={int(startTime)}&endTime={int(endTime)}&limit={int(limit)}'''  
    headers = {'content-type': 'application/json', 'X-MBX-APIKEY': api_key}
    response = requests.get(url + api_call, headers=headers)
    response_text = json.loads(response.text)
    current_API_weight = int(response.headers.get('x-mbx-used-weight-1m'))
    return(response_text)


def ws_CandlestickStream_trades(): 
    socket = f'wss://stream.binance.com:9443/ws/btceur@kline_1m'

    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)

    def on_error(wsapp,error):
        print(error)

    while True:  # Retry indefinitely until successful connection
        try:
            wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
            wsapp.run_forever()
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Retrying in 5 seconds...")
            time.sleep(5)  # Wait for 5 seconds before retrying
    
def handle_trades(json_message):
    # Here we should
    crypto_data  = []
    crypto_data.append(str(unix_to_datetime(json_message['E'])))
    crypto_data.append(str(json_message['s']))
    crypto_data.append(str(json_message['k']['o']))
    crypto_data.append(str(json_message['k']['c']))
    crypto_data.append(str(json_message['k']['h']))
    crypto_data.append(str(json_message['k']['l']))
    crypto_data.append(str(unix_to_datetime(json_message['k']['t'])))
    crypto_data.append(str(unix_to_datetime(json_message['k']['T'])))
    crypto_data.append(str(json_message['k']['v']))
    crypto_data.append(str(json_message['k']['n']))
    crypto_data.append(str(json_message['k']['q']))
    crypto_data.append(str(json_message['k']['x']))

    insert_data_candlesticks(castToDataType(crypto_data))
    print(crypto_data)
    print("-----------------------")

In [4]:
# We retrieve last 720 rows from API and load them into table

columns = ['timestamp',
            'open',
            'high',
            'low', 
            'close', 
            'volume', 
            'close_time', 
            'quote_asset_volume', 
            'number_of_trades', 
            'taker_buy_base_asset_volume', 
            'taker_buy_quote_asset_volume', 
            'unused_field']

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

current_time = int(time.time()*1000)
start_date_Unix = int(current_time - (720*60)*1000 )
end_date_Unix = int(current_time - 1000)
data = get_klines_data('BTCEUR', '1m',start_date_Unix,end_date_Unix,'720')

df_data = pd.DataFrame(data, columns=columns)

df_toInsert = pd.DataFrame({
    'symbol': 'BTCEUR',
    'event_time': df_data['timestamp'].apply(unix_to_datetime),
    'base_asset_volume': df_data['taker_buy_base_asset_volume'],
    'close': df_data['close'],
    'close_time': df_data['timestamp'] + 59*1000,  # Add 59 seconds to timestamp
    'high': df_data['high'],
    'kline_closed': 'True',
    'low': df_data['low'],
    'number_of_trades': df_data['number_of_trades'],
    'open': df_data['open'],
    'quote_asset_volume': df_data['quote_asset_volume'],
    'start_time': df_data['timestamp'].apply(unix_to_datetime)
})
df_toInsert['close_time'] = df_toInsert['close_time'].apply(unix_to_datetime)

df_toInsert.head()


# Establish a connection to the Cassandra cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

# Switch to your keyspace
session.set_keyspace('dmsb_tfm')

#Truncate tables before running
session.execute('TRUNCATE dmsb_tfm.candlesticks')
session.execute('TRUNCATE dmsb_tfm.candlesticks_pred')

for index, row in df_toInsert.iterrows():
    # Convert the row into a format suitable for insert_data_candlesticks function
    data = [
        row['event_time'],
        row['symbol'],
        row['open'],
        row['close'],
        row['high'],
        row['low'],
        row['start_time'],
        row['close_time'],
        row['base_asset_volume'],
        row['number_of_trades'],
        row['quote_asset_volume'],
        row['kline_closed']
    ]
    # Call the function to insert the data
    data = castToDataType(data)

    # Insert data
    insert_query = session.prepare("""
        INSERT INTO candlesticks (
            EVENT_TIME,
            SYMBOL,
            OPEN,
            CLOSE,
            HIGH,
            LOW,
            START_TIME,
            CLOSE_TIME,
            BASE_ASSET_VOLUME,
            NUMBER_OF_TRADES,
            QUOTE_ASSET_VOLUME,
            KLINE_CLOSED
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """)

    result = 'Data inserted'
    # Execute the insertion query
    try:
        session.execute(insert_query, data)
    except Exception as e:
        # print(f"An error occurred while inserting data: {e}")
        result = f'Error inserting data: {e}'

# Close the connection
cluster.shutdown()

In [5]:
ws_CandlestickStream_trades()

['2024-04-03 08:55:34', 'BTCEUR', '61856.88000000', '61874.68000000', '61882.29000000', '61856.88000000', '2024-04-03 08:55:00', '2024-04-03 08:55:59', '0.03136000', '6', '1940.34693210', 'False']
-----------------------
['2024-04-03 08:55:36', 'BTCEUR', '61856.88000000', '61870.77000000', '61882.29000000', '61856.88000000', '2024-04-03 08:55:00', '2024-04-03 08:55:59', '0.04175000', '7', '2583.18423240', 'False']
-----------------------
['2024-04-03 08:55:41', 'BTCEUR', '61856.88000000', '61867.52000000', '61882.29000000', '61856.88000000', '2024-04-03 08:55:00', '2024-04-03 08:55:59', '0.05673000', '10', '3509.95954630', 'False']
-----------------------
['2024-04-03 08:56:00', 'BTCEUR', '61856.88000000', '61867.52000000', '61882.29000000', '61856.88000000', '2024-04-03 08:55:00', '2024-04-03 08:55:59', '0.05673000', '10', '3509.95954630', 'True']
-----------------------
['2024-04-03 08:56:02', 'BTCEUR', '61890.69000000', '61890.69000000', '61890.69000000', '61890.69000000', '2024-04-